### Connect to google drive


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# !pip install missingno
# !pip install geopy

#Data Preperation  
##read 2019 Berline Airbnb Dataset
This file contain all the data of Berline Airbnb liatings from 2009 - 2019


In [5]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import warnings

# Ignore warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', 500)

df = pd.read_csv('/content/drive/MyDrive/kaggle/berline/Airbnb Berlin.csv')
pd.set_option('display.max_columns', None)
df['review_date'] = pd.to_datetime(df['review_date'])

df.shape

(456961, 47)

### filter the data
filter only 2019 , 2018 data

In [ ]:

# df = df.loc[df['review_date'].dt.year.isin([2019.0,2018.0])]
# df.head(100)

(224518, 46)

### Data Prepeartion

### feature selection  (round #1)
in this section we going to drop all the.
* redundant / duplicate data.
* drop all the column that contains same value , these data is not useful.
* drop columns that almost not containing any data.
* drop all the data the clearly will not help me during this jurney , like (Urls , reviewer  id , reviewer name ... )

In [6]:
df = df.drop(columns=['Review ID', 'Reviewer ID', 'Reviewer Name', 'Listing URL','Listing Name','Host ID', 'Host URL', 'Host Name',
                           'City', 'Country Code', 'Country','First Review', 'Last Review', 'Business Travel Ready',
            ])

# splite the review_date to day , month , year

In [ ]:
df['review_date'] = pd.to_datetime(df['review_date'])
df['year'] = df['review_date'].dt.year
# df['month'] = df['review_date'].dt.month
# df['day'] = df['review_date'].dt.day


In [ ]:
df['year'].value_counts()

,count
year,
2018.0,157685
2017.0,101522
2019.0,66833
2016.0,64059
2015.0,36789
2014.0,15928
2013.0,6661
2012.0,2704
2011.0,532


In [7]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 456961 entries, 0 to 456960
Data columns (total 33 columns):
 #   Column                Non-Null Count   Dtype         
---  ------                --------------   -----         
 0   index                 456961 non-null  int64         
 1   review_date           452805 non-null  datetime64[ns]
 2   Comments              452595 non-null  object        
 3   Listing ID            456961 non-null  int64         
 4   Host Since            456913 non-null  object        
 5   Host Response Time    398194 non-null  object        
 6   Host Response Rate    398194 non-null  object        
 7   Is Superhost          456913 non-null  object        
 8   neighbourhood         456961 non-null  object        
 9   Neighborhood Group    456961 non-null  object        
 10  Postal Code           449258 non-null  object        
 11  Latitude              456961 non-null  float64       
 12  Longitude             456961 non-null  float64       
 13 

#Fix and clean data

* remove from numbers the following extra chars , like $ , %  
* from catigories remove chars that not numbers  
* fix some of the Postal Code values that contains non relevant data



In [8]:
# remove the $ from price
df['Price'] = df['Price'].replace('[\$,]', '', regex=True).astype(float)

# remove the % from 'Host Response Rate'
df['Host Response Rate'] = df['Host Response Rate'].replace('[\%,]', '', regex=True).astype(float)

# Fix Postal Code incorrect values, remove '\n'and other irrelevant text
df['Postal Code'] = df['Postal Code'].astype(str).str[:5]

In [9]:
df.shape

(456961, 33)

# aggregate alls the reviews of Listing to one raw:
columns aggreagted on 3 deferent ways:
1. mean
2. sum
3. last value
4. String Concatination

In [10]:
# Define columns by type
mean_columns = ["Overall Rating","Accuracy Rating", "Cleanliness Rating", "Checkin Rating",
    "Communication Rating", "Location Rating", "Value Rating","Price","Square Feet"
    ]
sum_columns = ["Reviews"]
string_concat_values_columns = ["Comments"]
choose_last_value_columns = ["review_date","Bedrooms","Beds","Bathrooms","Host Response Rate","Is Superhost","Latitude","Longitude",
                         "Accomodates", "Guests Included","Min Nights","Instant Bookable",
                         "Accomodates", "Host Since","Is Exact Location","Postal Code", "Latitude", "Longitude","Property Type","Host Response Time", "Neighborhood Group","neighbourhood", "Room Type"
                         ]

# Aggregation functions
aggregation_functions = {
    **{col: "mean" for col in mean_columns},  # Mean for ratings
    **{col: "sum" for col in sum_columns},      # Sum for numerical values
    **{col: 'last' for col in choose_last_value_columns},  # Concat unique values
    **{col: lambda x: ", ".join(x.dropna().unique()) for col in string_concat_values_columns}  # Concat unique values
}

# Group by 'Listing ID' and apply aggregation
df_aggregated = df.groupby("Listing ID").agg(aggregation_functions).reset_index()

df_aggregated.head(2)

,Listing ID,Overall Rating,Accuracy Rating,Cleanliness Rating,Checkin Rating,Communication Rating,Location Rating,Value Rating,Price,Square Feet,Reviews,review_date,Bedrooms,Beds,Bathrooms,Host Response Rate,Is Superhost,Latitude,Longitude,Accomodates,Guests Included,Min Nights,Instant Bookable,Host Since,Is Exact Location,Postal Code,Property Type,Host Response Time,Neighborhood Group,neighbourhood,Room Type,Comments
0,2695,100.0,10.0,10.0,10.0,10.0,9.0,10.0,17.0,NaN,49,2019-04-21,1.0,1.0,1.0,50.0,f,52.54851,13.40455,2,1,2,f,09-16-08,t,10437,Apartment,within a day,Pankow,Prenzlauer Berg,Private room,I really enjoyed staying at Micha and Nadja's ...
1,3176,92.0,9.0,9.0,9.0,9.0,10.0,9.0,90.0,720.0,20736,2018-10-29,1.0,2.0,1.0,50.0,f,52.53500,13.41758,4,2,62,f,10-19-08,t,10405,Apartment,within a day,Pankow,Prenzlauer Berg,Entire home/apt,"excellent stay, i would highly recommend it. a..."


In [ ]:
df_aggregated.shape

(23536, 30)

#move out non relevant data for EDA to deferent datafram.
* now we will remove Comments,Longitude,Latitude out to and save it to deferent file for later use
* drop the Comments column from the main df


In [11]:
df_wiht_comments = df_aggregated[['Listing ID', 'Comments']].copy()
df_wiht_comments.to_csv('/content/drive/MyDrive/kaggle/berline/non_relevant_for_EDA.csv')
df_aggregated.drop(columns=['Comments'], inplace=True)

In [12]:
df_aggregated.describe()

,Listing ID,Overall Rating,Accuracy Rating,Cleanliness Rating,Checkin Rating,Communication Rating,Location Rating,Value Rating,Price,Square Feet,Reviews,review_date,Bedrooms,Beds,Bathrooms,Host Response Rate,Latitude,Longitude,Accomodates,Guests Included,Min Nights
count,2.353600e+04,18914.000000,18888.000000,18892.000000,18870.000000,18886.000000,18871.000000,18868.000000,23536.000000,425.000000,23536.000000,19380,23516.000000,23501.000000,23507.000000,13046.000000,23536.000000,23536.000000,23536.000000,23536.000000,23536.000000
mean,1.778264e+07,94.564344,9.679850,9.334904,9.728829,9.746479,9.550315,9.417638,69.612424,487.131765,1999.031824,2018-07-06 05:33:19.504643840,1.161720,1.641802,1.095971,91.842174,52.509906,13.406193,2.671737,1.346151,6.897774
min,2.695000e+03,20.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,0.000000,0.000000,0.000000,2011-08-22 00:00:00,0.000000,0.000000,0.000000,0.000000,52.345800,13.097180,1.000000,1.000000,1.000000
25%,9.056046e+06,92.000000,10.000000,9.000000,10.000000,10.000000,9.000000,9.000000,32.000000,0.000000,1.000000,2018-01-15 00:00:00,1.000000,1.000000,1.000000,95.000000,52.489177,13.375730,2.000000,1.000000,2.000000
50%,1.874818e+07,97.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,49.000000,420.000000,25.000000,2019-01-02 00:00:00,1.000000,1.000000,1.000000,100.000000,52.509210,13.416575,2.000000,1.000000,3.000000
75%,2.587987e+07,100.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,75.000000,753.000000,289.000000,2019-04-23 00:00:00,1.000000,2.000000,1.000000,100.000000,52.532670,13.439532,3.000000,1.000000,4.000000
max,3.468232e+07,100.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,9000.000000,4639.000000,297025.000000,2019-05-14 00:00:00,10.000000,22.000000,8.500000,100.000000,52.651670,13.757640,16.000000,16.000000,1000.000000
std,9.887995e+06,7.599949,0.737798,1.031248,0.695275,0.679600,0.748713,0.840456,216.660408,517.652098,9922.658422,NaN,0.660736,1.228642,0.346150,19.447852,0.031279,0.058574,1.567812,0.856297,24.879032


### From the Describe above we can see the following :
**Price:** there Zero values and big values like 9000 for a night , these values should be remove during the outliers section

**Reviews:** there are Listings with allmost 300k review , this is not real number that can bias the prediction , so we need to remove it

**Bedrooms:** there are Zero bedrooms , we need to fix

**Beds:** there are Zero bedrooms , we need to fix

**Min Nights:** 1000 min nights is not real number , need a fix


In [13]:
df_aggregated.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23536 entries, 0 to 23535
Data columns (total 31 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   Listing ID            23536 non-null  int64         
 1   Overall Rating        18914 non-null  float64       
 2   Accuracy Rating       18888 non-null  float64       
 3   Cleanliness Rating    18892 non-null  float64       
 4   Checkin Rating        18870 non-null  float64       
 5   Communication Rating  18886 non-null  float64       
 6   Location Rating       18871 non-null  float64       
 7   Value Rating          18868 non-null  float64       
 8   Price                 23536 non-null  float64       
 9   Square Feet           425 non-null    float64       
 10  Reviews               23536 non-null  int64         
 11  review_date           19380 non-null  datetime64[ns]
 12  Bedrooms              23516 non-null  float64       
 13  Beds            

we can see from the dataframe info , there are object colummns that represent number / catigories.
there is a need to transform these objects to numbers


# Data uniquness

In [14]:
df_aggregated.nunique().sort_values(ascending=False)

,0
Listing ID,23536
Longitude,13260
Latitude,10032
Host Since,3085
review_date,1464
Reviews,332
Price,326
Postal Code,207
Square Feet,109
Min Nights,97


#Missing data

In [15]:
missing_values = df_aggregated.isna().sum()
missing_values = missing_values[missing_values > 0]
# Display results
print(missing_values)

Overall Rating           4622
Accuracy Rating          4648
Cleanliness Rating       4644
Checkin Rating           4666
Communication Rating     4650
Location Rating          4665
Value Rating             4668
Square Feet             23111
review_date              4156
Bedrooms                   20
Beds                       35
Bathrooms                  29
Host Response Rate      10490
Is Superhost               21
Host Since                 21
Host Response Time      10490
dtype: int64


in the above we can see the each column that have missing values need to address:

**Square Feet:** almost empty column , need to remove it

**Host Response Rate , Host Response Time:** allmost halph missing

**Overall Rating,Accuracy Rating,Cleanliness Rating,Checkin Rating,Communication Rating,Location Rating,Value Rating,review_date:** have ~20%-30% missing data


## from the unique values we can learn:

there are 3 type of columns

* column with high Cardinality like Liting Id , Comments, Latitudes , there is a need to remove as much as we can fromthese columns

* columns with two values like Is SuperHost, Is Exact Locaion , Instant Bookable , apparently these are boolean , need to transorm to numbers

* columns with multiple values from 3 - 50 , these columns contains catigories , we need to narrow these catigories to samll one




# wide catigories Reduce :
* Host Response Rate Grouped
* Overall Rating
* Neighbourhood Grouped
* property_types


In [16]:
# Define bins and labels for 'Host Response Rate'
bins = [0, 50, 80, 95, 100]
labels = ["Low Response", "Moderate Response", "High Response", "Very High Response"]

# Apply pd.cut() to create a new binned column
df_aggregated["Host Response Rate Cleansed"] = pd.cut(df_aggregated["Host Response Rate"], bins=bins, labels=labels, include_lowest=True)

#  Reduce Categories
df_aggregated['Overall Rating Cleansed'] = df_aggregated['Overall Rating'].apply(lambda x: np.ceil(x/10))


#2. Grouping neighbourhoods into Neighborhood Groups
neighbourhood_mapping = df_aggregated.groupby('neighbourhood')['Neighborhood Group'].first()
df_aggregated['Neighbourhood Grouped Cleansed'] = df_aggregated['neighbourhood'].map(neighbourhood_mapping)

# 3. Reducing Property Types
property_mapping = {
    "Villa": "Vacation Rental",
    "Cottage": "Vacation Rental",
    "Bungalow": "Vacation Rental",
    "Cabin": "Vacation Rental",
    "Tiny house": "Vacation Rental",
    "Earth house": "Vacation Rental",
    "Treehouse": "Vacation Rental",
    "Hut": "Vacation Rental",
    "Barn": "Vacation Rental",
    "Houseboat": "Boats & Houseboats",
    "Boat": "Boats & Houseboats",
    "Camper/RV": "Mobile/Alternative Lodging",
    "Cave": "Mobile/Alternative Lodging",
    "Pension (South Korea)": "Mobile/Alternative Lodging",
    "Casa particular (Cuba)": "Mobile/Alternative Lodging",
}

# Apply mapping and assign 'Other' to rare categories
top_property_types = [
    "Apartment", "Loft", "House", "Townhouse", "Condominium", "Serviced apartment",
    "Hotel", "Hostel", "Guesthouse", "Bed and breakfast", "Boutique hotel"
]

df_aggregated['Property Type Cleansed'] = df_aggregated['Property Type'].apply(
    lambda x: property_mapping.get(x, x) if x in top_property_types or x in property_mapping else "Other"
)

# 4. Binning Postal Codes (first two digits represent broad area)
# df_aggregated['Postal Code Cleansed'] = df_aggregated['Postal Code'].astype(str).str[:2]  # Use only first 2 digits

### Transform data:

In [ ]:
df_aggregated.shape

(23536, 35)

#Transforming Data
* booleans to True/False

In [24]:
#transform true/false into bool
df_aggregated['Instant Bookable'] = df_aggregated['Instant Bookable'].replace({'t': True, 'f': False})
df_aggregated['Instant Bookable'] = df_aggregated['Instant Bookable'].astype(bool)

df_aggregated['Is Exact Location'] = df_aggregated['Is Exact Location'].replace({'t': True, 'f': False})
df_aggregated['Is Exact Location'] = df_aggregated['Is Exact Location'].astype(bool)

df_aggregated['Is Superhost'] = df_aggregated['Is Superhost'].replace({'t': True, 'f': False})
df_aggregated['Is Superhost'] = df_aggregated['Is Superhost'].astype(bool)

In [23]:
df_aggregated['Is Superhost']

,Is Superhost
0,False
1,False
2,True
3,False
4,False
...,...
23531,False
23532,False
23533,False
23534,False


In [ ]:
# df_aggregated.drop(columns=['Host Since',
#                  'Is Exact Location',"Host Response Rate","Overall Rating","Instant Bookable","Is Superhost","Instant Bookable","Property Type"
#                  ], inplace=True)

In [ ]:
df_aggregated.shape

(23536, 35)

In [18]:
df_aggregated.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23536 entries, 0 to 23535
Data columns (total 35 columns):
 #   Column                          Non-Null Count  Dtype         
---  ------                          --------------  -----         
 0   Listing ID                      23536 non-null  int64         
 1   Overall Rating                  18914 non-null  float64       
 2   Accuracy Rating                 18888 non-null  float64       
 3   Cleanliness Rating              18892 non-null  float64       
 4   Checkin Rating                  18870 non-null  float64       
 5   Communication Rating            18886 non-null  float64       
 6   Location Rating                 18871 non-null  float64       
 7   Value Rating                    18868 non-null  float64       
 8   Price                           23536 non-null  float64       
 9   Square Feet                     425 non-null    float64       
 10  Reviews                         23536 non-null  int64         
 11  re

In [25]:
df_aggregated.to_pickle("/content/drive/MyDrive/kaggle/berline/ready_for_eda.pkl")